In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# loading data
a = pd.read_excel("3.xlsx")

In [3]:
#preventing from frequent loading 
df = a

In [4]:
#drop rows with at least 30 missing values
df.dropna(thresh=df.shape[1] - 30, inplace=True)

In [5]:

# Drop column: 'L1_BloodGroup_First'
df = df.drop(columns=['L1_BloodGroup_First'])
# Drop rows with missing data in column: 'Outcome_InhospitalMortality'
df = df.dropna(subset=['Outcome_InhospitalMortality'])


In [6]:
# splitting internal and external validation
df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]
df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])

In [7]:
df_internal_validation

,Outcome_InhospitalMortality,Outcome_ICUadmission,TM_S_Intubation,TM_S_Dialysis,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,Symptom_Chiver,Symptom_Mylagia,...,LAB_CR_1,LAB_NA_First,LAB_K_First,LAB_ALKP_First,LAB_ESR_First,LAB_CPK_First,LAB_PTT_First,LAB_PT_First,LAB_INR_First,Demographic_Age
0,0.0,1,1,1,1.0,1,1,0,0,0,...,NaN,140.0,4.0,181.0,62.0,661.0,32.6,10.9,1.00,33.0
1,0.0,1,1,0,1.0,1,1,0,0,1,...,1.1,139.0,3.5,98.0,30.0,179.0,37.6,10.8,0.99,44.0
2,0.0,1,0,1,1.0,0,1,0,0,1,...,0.9,138.0,4.2,99.0,29.0,1956.0,27.4,12.1,1.11,50.0
3,1.0,1,0,0,1.0,0,1,1,0,1,...,1.2,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,71.0
4,1.0,1,0,0,1.0,0,1,1,0,1,...,1.2,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,1.0,0,0,0,0.0,1,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0
9130,1.0,0,0,0,1.0,1,1,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.0
9131,1.0,1,0,0,0.0,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0
9132,1.0,0,0,0,1.0,1,1,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0


In [8]:
X = df_internal_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y  = df_internal_validation['Outcome_InhospitalMortality']

In [9]:
X_ex = df_external_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_ex  = df_external_validation['Outcome_InhospitalMortality']

In [10]:
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.2 , random_state = 42)

In [11]:
num =  [
    "symtpm_to_referral",
    "VS_O2satwithoutsupp",
    "VS_PR",
    "VS_diastolic BP",
    "VS_Systolic BP",
    "VS_RR",
    "VS_T",
    "LAB_WBC_1",
    "LAB_LYMPHH_1",
    "LAB_NEUT_1",
    "LAB_PLT_1",
    "LAB_HB_1",
    "LAB_MCV_1",
    "LAB_CR_1",
    "LAB_NA_First",
    "LAB_K_First",
    "LAB_ALKP_First",
    "LAB_ESR_First",
    "LAB_CPK_First",
    "LAB_PTT_First",
    "LAB_PT_First",
    "LAB_INR_First",
    "Demographic_Age"
]

In [12]:
cat = X.drop(columns= num).columns.tolist()

In [13]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

X_train_cat = imputer.fit_transform(np.array(X_train[cat]))
X_test_cat = imputer.fit_transform(np.array(X_test[cat]))
X_ex_cat = imputer.fit_transform(np.array(X_ex[cat]))

X_train_cat = pd.DataFrame(X_train_cat, columns=cat)
X_test_cat = pd.DataFrame(X_test_cat, columns=cat)
X_ex_cat = pd.DataFrame(X_ex_cat, columns=cat)

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)

X_train_num = imputer.fit_transform(X_train[num])
X_test_num = imputer.fit_transform(X_test[num])
X_ex_num = imputer.fit_transform(X_ex[num])

X_train_num = pd.DataFrame(X_train_num, columns=num)
X_test_num = pd.DataFrame(X_test_num, columns=num)
X_ex_num = pd.DataFrame(X_ex_num, columns=num)


d:\python\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
d:\python\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [15]:
X_train = pd.concat([X_train_cat, X_train_num], axis=1)
X_test = pd.concat([X_test_cat, X_test_num], axis=1)
X_ex = pd.concat([X_ex_cat, X_ex_num], axis=1)


In [16]:
from sklearn.ensemble import RandomForestClassifier

def random_forest_feature_selection(X_train, y_train, X_test, X_ex, num_features=40, random_state=42):
    # Initialize and fit a Random Forest classifier
    clf = RandomForestClassifier(random_state=random_state)
    clf.fit(X_train, y_train)

    # Get feature importances
    feature_importances = clf.feature_importances_

    # Sort features by their importance
    sorted_indices = feature_importances.argsort()[::-1]

    # Select top features
    selected_feature_indices = sorted_indices[:num_features]

    # Filter the features
    X_train_selected = X_train.iloc[:, selected_feature_indices]
    X_test_selected = X_test.iloc[:, selected_feature_indices]
    X_ex_selected = X_ex.iloc[:, selected_feature_indices]

    return X_train_selected, X_test_selected, X_ex_selected, selected_feature_indices
#X_train,X_test,X_ex, selected_feature_indices = random_forest_feature_selection(X_train, y_train,X_test,X_ex)

In [17]:
from sklearn.linear_model import Lasso

def lasso_MO_ICU(X_train, y_train,X_test , X_ex):

    
    lasso = Lasso(alpha=0.0001,random_state=42 )
    lasso.fit(X_train, y_train)


    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_ex = pd.DataFrame(X_ex)

    selected_feature_indices = sorted_indices[:20]
    selected_feature_indices = np.where(lasso.coef_ != 0)[0]


    X_train_selected_Mortality_ICU = X_train.iloc[:, selected_feature_indices]
    X_test_selected_Mortality_ICU = X_test.iloc[:, selected_feature_indices]
    X_ex = X_ex.iloc[:, selected_feature_indices]
    return X_train_selected_Mortality_ICU ,X_test_selected_Mortality_ICU,X_ex, selected_feature_indices

X_train,X_test,X_ex, selected_feature_indices = lasso_MO_ICU(X_train, y_train,X_test,X_ex)


d:\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.134e+01, tolerance: 9.875e-02
  model = cd_fast.enet_coordinate_descent(


In [18]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

transform = MinMaxScaler()

def standard(f):
    df_ex_normalize = transform.fit_transform(f)
    df_ex_normalize = pd.DataFrame(df_ex_normalize)
    df_ex_normalize.columns = f.columns
    return df_ex_normalize


In [19]:
X_train[num]=standard(X_train[num])
X_test[num]=standard(X_test[num])
X_ex[num]=standard(X_ex[num])

In [20]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def calculate_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    cm = confusion_matrix(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted)
    recall = recall_score(y_test, y_predicted)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) != 0 else 0
    f1 = f1_score(y_test, y_predicted)
    
    return accuracy, precision, recall, specificity, f1

In [22]:
from sklearn.metrics import roc_auc_score

def calculate_and_update_model_metrics(y_test, y_predicted,y_ex, y_ex_predicted):
    accuracy, precision, recall, specificity, f1 = calculate_metrics(y_test, y_predicted)
    model = {}
    model["accuracy"] = accuracy
    model["precision"] = precision
    model["recall"] = recall
    model["specificity"] = specificity
    model["f1"] = f1
    model["AUC"] = roc_auc_score(y_test, y_predicted)
    
    accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex = calculate_metrics(y_ex, y_ex_predicted)
    model["accuracy_ex"] = accuracy_ex
    model["precision_ex"] = precision_ex
    model["recall_ex"] = recall_ex
    model["specificity_ex"] = specificity_ex
    model["f1_ex"] = f1_ex
    model["AUC_ex"] = roc_auc_score(y_test, y_predicted)
    return model


In [23]:
def logistic_regression_classifier(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = LogisticRegression(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)

    

    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [24]:
def train_and_evaluate_svm(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = svm.SVC(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)



    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [25]:
def train_and_evaluate_tree(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = DecisionTreeClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [26]:
def train_and_evaluate_knn(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {}
    lr = KNeighborsClassifier()
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [27]:
def train_and_evaluate_forest(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = RandomForestClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)


    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [28]:
from xgboost import XGBClassifier
def train_and_evaluate_boost(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}
    lr = XGBClassifier(random_state=42)
    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_pred_test = best_classifier_lr.predict(X_test)



    

    y_pred_external = best_classifier_lr.predict(X_ex)
    Logistic_Regression =  Logistic_Regression =  calculate_and_update_model_metrics(y_test, y_pred_test,y_ex, y_pred_external)
    
    
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_pred_test
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_pred_external
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [29]:
list_Outcome_InhospitalMortality=[]




In [30]:
y_train_ = np.array(y_train)
y_test_ = np.array(y_test)

y_ex_ = np.array(y_ex)




In [31]:

list_Outcome_InhospitalMortality.extend([logistic_regression_classifier(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                            train_and_evaluate_svm(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_tree(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_knn(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_forest(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_boost(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_)])


result_dic_list_Outcome_InhospitalMortality = dict(zip(['logistic_regression', 'svm', 'tree', 'knn', 'forest', 'boost'], list_Outcome_InhospitalMortality))





merged_dict = {}

# List of dictionary names and their corresponding dictionaries
dict_list = [('Outcome_InhospitalMortality', result_dic_list_Outcome_InhospitalMortality)]

# Merge the dictionaries
for name, result_dict in dict_list:
    merged_dict[name] = result_dict

# The merged_dict now contains all the dictionaries merged together


dff = pd.DataFrame(merged_dict)


dff = dff.transpose()


dff.reset_index(inplace=True)
dff.rename(columns={'index': 'Method'}, inplace=True)





data = []

for outcome, models in merged_dict.items():
    for model, metrics in models.items():
        accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex = metrics[0]['accuracy'], metrics[0]['precision'], metrics[0]['recall'], metrics[0]['specificity'], metrics[0]['f1'], metrics[0]['AUC'],metrics[0]['accuracy_ex'], metrics[0]['precision_ex'], metrics[0]['recall_ex'], metrics[0]['specificity_ex'], metrics[0]['f1_ex'], metrics[0]['AUC_ex']
        y_true, y_predicted,y_true_ex,y_predicted_ex = metrics[1]['y_true'], metrics[1]['y_predicted'],metrics[1]['y_true_ex'], metrics[1]['y_predicted_ex']
        data.append([outcome, model,accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex, y_true.tolist(), y_predicted.tolist(),y_true_ex.tolist(),y_predicted_ex.tolist()])

columns = ['Outcome', 'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1','AUC', 'Accuracy_ex', 'Precision_ex', 'Recall_ex', 'Specificity_ex', 'F1_ex','AUC_ex', 'y_true', 'y_predicted', 'y_true_ex', 'y_predicted_ex']

dff = pd.DataFrame(data, columns=columns)



d:\python\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
dff

,Outcome,Model,Accuracy,Precision,Recall,Specificity,F1,AUC,Accuracy_ex,Precision_ex,Recall_ex,Specificity_ex,F1_ex,AUC_ex,y_true,y_predicted,y_true_ex,y_predicted_ex
0,Outcome_InhospitalMortality,logistic_regression,0.318301,0.984848,0.157895,0.989831,0.272156,0.573863,0.506742,0.949686,0.403025,0.915789,0.565896,0.573863,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Outcome_InhospitalMortality,svm,0.808497,0.836910,0.947368,0.227119,0.888720,0.587244,0.804826,0.816083,0.975089,0.133333,0.888529,0.587244,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,Outcome_InhospitalMortality,tree,0.702614,0.844523,0.774089,0.403390,0.807774,0.588739,0.432931,0.826962,0.365658,0.698246,0.507094,0.588739,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
3,Outcome_InhospitalMortality,knn,0.791503,0.827143,0.937652,0.179661,0.878937,0.558656,0.783534,0.809992,0.951957,0.119298,0.875256,0.558656,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
4,Outcome_InhospitalMortality,forest,0.764706,0.866723,0.837247,0.461017,0.851730,0.649132,0.657913,0.842949,0.701957,0.484211,0.766019,0.649132,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, ..."
5,Outcome_InhospitalMortality,boost,0.737255,0.862489,0.802429,0.464407,0.831376,0.633418,0.735983,0.830986,0.839858,0.326316,0.835398,0.633418,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
